In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

In [ ]:
D = pd.read_csv('results_test.log')
D.columns

In [ ]:
D[(D.alpha==0.01) & (D.dt==0.01)].plot(x='s2', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                      style='o', logx=True, logy=True,
                      title='alpha=0.01, dt=0.01');

In [ ]:
D[D.s2==1.0].plot(x='alpha', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                  style='o', logx=True, logy=False, 
                  title='s2=s3 = 1.0');

In [ ]:
D[D.s2==0.2].plot(x='alpha', y=['Te_0_mean_50%', 'Te_0_std_50%'], 
                  style='o', logx=True, logy=False, 
                  title='s2=s3 = 0.2');

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
for i, g in D[D.s2==0.2].groupby('dt'):
    g.plot(x='alpha', y='Te_0_std_50%', 
           style='o', logx=True, logy=True, ax=ax, label='dt=%s' % (i), 
           title='s2=s3 = 0.2')
plt.ylabel('Te_0_std_50%');

In [ ]:
D[D.s2==0.2][['s2', 's3', 'alpha', 'beta', 'gamma', 'dt', 'NITER', 'Te_0_mean_50%', 'Te_0_std_50%']]